<center>
<img src="logo.png" height="900"> 
</center>


#  Анализируем чеки

В этом задании мы будем работать с покупками и чеками. Смотреть за корреляциями в покупках довольно полезно.

> В 1992 году группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (Drug Store — формат разнокалиберных магазинов у дома). После анализа всех этих транзакций самым сильным правилом получилось «Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе». К сожалению, такое правило показалось руководству Osco Drug настолько контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали. Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с работы домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайший магазин. И мужья, не долго думая, совмещали приятное с полезным — покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения.

Для работы будем использовать датасет о продуктовых корзинах: https://www.kaggle.com/heeraldedhia/groceries-dataset

In [2]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline
COLLAB = False

Подружаем данные и смотрим как они выглядят.

In [3]:
if COLLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    df = pd.read_csv("/content/gdrive/MyDrive/DA/2/groceries.csv", sep=',')
else:
    df = pd.read_csv('data/groceries.csv', sep=',')
df.columns = ['id', 'fielddate', 'product']
print(df.shape)
df.head()

Mounted at /content/gdrive
(38765, 3)


,id,fielddate,product
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


## 1. Корреляции

Для начала поработаем с корреляциями в данных. 

__а)__ Какой товар покупался чаще всего? Сохраните название этого товара в переменную `product_name`.

In [4]:
product_name = df['product'].value_counts().index[0]
print(product_name)

whole milk


In [ ]:
# проверка, что задание решено корректно
assert len(product_name) == 10

__б)__ Сколько всего уникальных заказов было сделано? Сохраните число заказов в переменную `n_cnt`.

In [8]:
n_cnt = df['id'].unique().shape[0]
print(n_cnt)

3898


In [9]:
# проверка, что задание решено корректно
assert n_cnt > 3800
assert n_cnt < 4000

В таблице выше в каждой строке записана информация о покупке конкретного товара. Давайте сделаем табличку размера "число товаров" на "число покупок", чтобы понимать какие товары покупались вместе, а какие нет. 

> Обратите внимание, то здесь задание немного упрощено. Вообще говоря, нам нужно делать агрегацию по паре `fielddate, id`, если мы хотим изучать чеки по-честному. Но мы делаем её только по `id` для того, чтобы не усложнять задание. В качестве необязательного дополнения вы можете после сдачи задания переделать код так, чтобы дата тоже учитывалась при расчётах. 

In [10]:
sparse_sales = pd.pivot_table(df, 
               values='fielddate', 
               index='id', 
               columns='product', 
               fill_value=0, aggfunc='count')

sparse_sales.head()

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
id,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,1,0
1001,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,2,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0


В нашей матрице огромное число нулей. Обычно такие матрицы называют разряжеными. Мы занимаем нулями кучу свободной памяти, которую мы могли бы не занимать, если бы хранили данные [в ином виде.](https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/)

__в)__ Постройте матрицу корреляций Пирсона. Для этого используйте метод таблицы `.corr`.

In [11]:
sales_correlation = sparse_sales.corr(method='pearson')
sales_correlation.head()

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
product,,,,,,,,,,,,,,,,,,,,,
Instant food products,1.000000,-0.006936,-0.009420,-0.010825,-0.003470,-0.004007,-0.010419,-0.008275,0.005847,-0.014528,...,0.025355,0.021851,0.005348,0.014025,-0.005670,0.016640,0.002853,0.015981,0.005071,0.018221
UHT-milk,-0.006936,1.000000,0.013806,0.006105,-0.007877,0.018349,0.009462,-0.018785,0.002897,0.020801,...,-0.003803,-0.024466,0.009144,0.006618,-0.012871,-0.011550,0.009990,0.028747,0.006505,0.028753
abrasive cleaner,-0.009420,0.013806,1.000000,-0.006523,-0.002091,-0.002415,-0.013429,-0.004986,0.018970,0.001813,...,0.012922,-0.008507,-0.007390,0.002163,-0.003417,0.010777,0.000107,0.003558,0.008360,-0.009420
artif. sweetener,-0.010825,0.006105,-0.006523,1.000000,0.105251,-0.002775,-0.015432,-0.005730,-0.006645,0.016042,...,0.049392,-0.009776,0.010179,-0.006614,-0.003926,-0.006806,-0.018394,0.029591,-0.024397,-0.010825
baby cosmetics,-0.003470,-0.007877,-0.002091,0.105251,1.000000,-0.000889,-0.004947,-0.001837,0.014798,-0.007983,...,-0.003919,-0.003134,-0.007413,-0.011288,-0.001259,-0.008476,-0.005896,-0.021056,-0.000410,-0.003470


Какие продукты сильнее всего коррелируют с яйцами, `domestic eggs` (их чаще всего покупают вместе)?  Сохраните название самого скоррелированного продукта в переменную `top_1`.

In [13]:
top_1 = sales_correlation['domestic eggs'].sort_values().index[-2]
print(top_1)

meat spreads


Какие продукты "мешают" купить яйца, то есть отрицательно коррелируют с их покупкой? Сохраните название продукта с самой большой отрицательной корреляцией в переменную `bottom_1`.

In [15]:
bottom_1 = sales_correlation['domestic eggs'].sort_values().index[0]
print(bottom_1)

pet care


In [ ]:
# проверка, что задание решено корректно
assert len(bottom_1) == 8
assert len(top_1) == 12

Напишите код, который выводит самые коррелируемые товары для случайного продукта из списка `unique_products`.

In [17]:
from random import choice
unique_products = df['product'].unique()
product = choice(unique_products)
high_corr = sales_correlation[product].sort_values().index[-2]
print(high_corr,"+", product)

bags + liver loaf


__г)__ Какие два продукта коррелируют сильнее всего? Положите их название в лист `answer`

In [22]:
max_corr = sales_correlation[sales_correlation != 1].max().sort_values(ascending=False)[0:2].index
answer = max_corr.values
print(answer)

['preservation products' 'soups']


In [23]:
# проверка, что задание решено корректно
assert 'soups' in answer

Конечно же, корреляция — это [не единственный способ искать](https://habr.com/ru/company/ods/blog/353502/) между покупками ассоциативные правила.

## 2. Зависимость. 

В лекции мы с вами сказали, что события $A$ и $B$ называются независимыми, если $P(AB) = P(A)\cdot P(B)$. Отталкиваясь от этого определения, можно ввести другую характеристику, которая показывает, насколько продукты зависят друг от друга, а именно __поддержку (lift).__ 

$$
lift = \frac{P(AB)}{P(A)\cdot P(B)}
$$

Эта метрика описывает отношение зависимости товаров к их независимости. Если оказалось, что `lift = 1`, это означает, что покупка товара $A$ не зависит от покупки товара $B$. Если `lift > 1`, то это означает, что вероятность встретить оба товара в чеке, $P(AB)$ высокая, то есть товары покупают вместе. Если `lift < 1`, это означает, что товары, наоборот, очень часто покупают по-отдельности. 

__д)__ Посчитайте значение нашей метрики для яиц и молока (`'whole milk', 'domestic eggs'`). Запишите получившиеся значение метрики в переменную `answer`.

> Вам аккуратно нужно сделать три среза по условию  `>= 1`. Там, где пара надо делать срез так, чтобы оба товара дали `True`. Сделать это в одну строку вам поможет метод `.all(axis=1)`. Частоты можно получить методом `.mean()`, так как python думает, что `False` - это ноль, а `True` - это единица.

In [25]:
pab= sparse_sales[(sparse_sales["domestic eggs"]>=1)&(sparse_sales["whole milk"]>=1)]["whole milk"].count()/sparse_sales["whole milk"].count()
papb = sparse_sales[sparse_sales["whole milk"]>=1]["whole milk"].count() * sparse_sales[sparse_sales["domestic eggs"]>=1]["whole milk"].count()/sparse_sales["whole milk"].count()**2
answer = pab / papb
print(answer)

1.152241691425711


In [26]:
# проверка, что задание решено корректно
assert answer < 3
assert answer > 1

__е)__ Посчитайте значение метрики для всех пар продуктов из датасета. Сохраните значения в словарик `dict`. В качестве ключа используете кортеж из пары продуктов. Чтобы удобнее было перебрать все сочетания, используйте `combinations` из модуля `itertools`.

Чтобы при подсчётах не возникало деления на ноль, добавьте к знаменателю маленькое число, например `1e-10`.

In [28]:
def metric(first, second):
    pab = sparse_sales[(sparse_sales[first]>=1)&(sparse_sales[second]>=1)][second].count()/sparse_sales[second].count()
    papb = sparse_sales[sparse_sales[second]>=1][second].count() * sparse_sales[sparse_sales[first]>=1][second].count()/sparse_sales[second].count()**2
    return pab / papb

In [29]:
import itertools 
paires = list(itertools.combinations(list(sparse_sales.columns), 2))
lift = {}
for pair in paires:
    lift[pair] = metric( pair[0],  pair[1])

Сколько пар продуктов покупали вместе хотя бы раз? Запишите ответ в переменную `answer`.

In [39]:
answer = 0
for key in lift.keys():
    if lift[key] > 0:
        answer += 1 
print(answer)

9824


Для какой пары продуктов метрика $lift$ оказалась самой большой? 

In [36]:
max_value = max(lift.values())
for key in lift.keys():
    if lift[key] == max_value:
        answer_1 = key
        break
print(answer_1)

('preservation products', 'soups')


Сколько раз эти продукты встретились в выборке? Как думаете адеватно ли делать выводы по такому объёму данных? 

In [ ]:
print(sparse_sales[sparse_sales['soups'] >= 1]['soups'].count())
print(sparse_sales[sparse_sales['preservation products'] >= 1]['preservation products'].count())

**ОТВЕТ:** Нет, объем данных слишком мал.

Для какой пары продуктов метрика оказывается самой маленькой? 

In [37]:
min_value = min(lift.values())
for key in lift.keys():
    if lift[key] == min_value:
        answer_2 = key
        break
print(answer_2)

('Instant food products', 'abrasive cleaner')


In [ ]:
print(sparse_sales[sparse_sales['Instant food products'] >= 1]['Instant food products'].count())
print(sparse_sales[sparse_sales['abrasive cleaner'] >= 1]['abrasive cleaner'].count())

In [40]:
# проверка, что задание решено корректно

assert answer < 10000
assert answer > 9000

## 3. Неоцениваемые задания

Выше мы увидели, что некоторые продукты встречаются в выборке очень редко. Понятное дело, что по ним у нас не получится построить хорошее ассоциативное правило. Попробуйте повторить расчёт той же метрики, но с условием что продукт покупали больше 10 раз. Изучите самые покупаемые вместе продукты и самые непокупаемые вместе продукты. Насколько сильно список отличается от полученного в предыдущем задании? 

In [44]:
print(sparse_sales.shape)
for product in sparse_sales.columns:
    counter = sparse_sales[sparse_sales[product] >= 1][product].count()
    if counter < 10:
        sparse_sales = sparse_sales.drop(columns = product)
print(sparse_sales.shape)

(3898, 167)
(3898, 154)


In [45]:
paires = list(itertools.combinations(list(sparse_sales.columns), 2))
lift = {}
for pair in paires:
    lift[pair] = metric( pair[0],  pair[1])

In [46]:
max_value = max(lift.values())
for key in lift.keys():
    if lift[key] == max_value:
        answer_1 = key
        break
print("Max lift pair is", answer_1)

('flower soil/fertilizer', 'organic products')


In [47]:
min_value = min(lift.values())
for key in lift.keys():
    if lift[key] == min_value:
        answer_2 = key
        break
print("Min lift pair is", answer_2)

('Instant food products', 'abrasive cleaner')


Иногда в чеках пытаются искать __продукты-якоря.__ То есть продукты, которые являются основными. Например: айфон - основной продукт, наушники и чехол - дополнения к нему. Подумайте как можно попытаться найти такие продукты на основе простых метрик, основанных на подсчёте условных вероятностей.

**ОТВЕТ:** Продукты не якори будут приобретаться преимущественно при условии наличия в чеке продукта якоря. Т.е. вероятность покупки аксессуаров вместе с айфоном будет значительно больше, чем без него $P(A|B) >> P(A)$. Можно попробовать какую-то метрику типа $$\frac{P(A|B)}{P(A)}$$, чем она больше, тем менее самостоятельный продукт А